## Wind Farm B Code 2 (Moderate for Middle Row Turbines)

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve
import os

# Dataset IDs
dataset_ids = [21, 23, 27, 53, 77, 82, 86, 87] ## According to determined tubrine layout, these datasets belong to middle row

# File paths
input_dir = r"D:\Master Thesis New Data Set\CARE DATA SET\CARE_To_Compare\Wind Farm B\Wind Farm B\datasets"
output_dir = r"D:\Master Thesis New Data Set\Final DataSet\Wind Farm B Code 2"
os.makedirs(output_dir, exist_ok=True)

def calculate_relative_wind_direction(awd, nd):
    return (awd - nd + 540) % 360 - 180

def get_ideal_power(wind_speed, cut_in, rated_speed, rated_power):
    if wind_speed < cut_in:
        return 0
    elif wind_speed < rated_speed:
        return rated_power * ((wind_speed - cut_in) / (rated_speed - cut_in))**3
    else:
        return rated_power

for dataset_id in dataset_ids:
    try:
        print(f"\n🔄 Processing dataset {dataset_id}...")

        # Load data
        file_path = os.path.join(input_dir, f"{dataset_id}.csv")
        df = pd.read_csv(file_path, delimiter=';')

        # Set negative power to 0
        df['power_62_avg'] = df['power_62_avg'].apply(lambda x: max(x, 0))

        # Calculate relative wind direction
        df['relative_wind_direction'] = calculate_relative_wind_direction(df['sensor_4_avg'], df['sensor_21_avg'])

        # Step 1: Filter for normal operation
        df_filtered = df[
            (df['sensor_10_avg'] <= 2) & 
            (df['sensor_10_avg'] >= -5) & 
            (df['relative_wind_direction'] >= -10) & 
            (df['relative_wind_direction'] <= 10) & 
            (df['status_type_id'] == 0) &
            (df['train_test'] == 'train')
        ].dropna(subset=['wind_speed_61_avg', 'power_62_avg'])

        df_sorted = df_filtered.sort_values(by='wind_speed_61_avg')

        # Normalize power
        original_max_power = df_sorted['power_62_avg'].max()
        df_sorted['power_62_avg'] = df_sorted['power_62_avg'] / original_max_power

        # Rated region (normalized)
        rated_band = df_sorted[
            (df_sorted['power_62_avg'] >=  0.98) &
            (df_sorted['power_62_avg'] <= 1.02)
        ]
        rated_band_capped = rated_band[
            (rated_band['wind_speed_61_avg'] >= 10) & 
            (rated_band['wind_speed_61_avg'] <= 12)
        ]
        rated_speed = rated_band_capped['wind_speed_61_avg'].min()
        rated_power = 1  # normalized

        # Cut-in estimation
        wind_speed_3_df = df_sorted[
            (df_sorted['wind_speed_61_avg'] >= 2.98) &
            (df_sorted['wind_speed_61_avg'] <= 3.02)
        ]
        min_power_at_3 = wind_speed_3_df['power_62_avg'].min()

        def equation(cut_in):
            return rated_power * ((3 - cut_in) / (rated_speed - cut_in))**3 - min_power_at_3

        cut_in = fsolve(equation, 1.0)[0]

        # Normalize full data
        df['power_62_avg_norm'] = df['power_62_avg'] / original_max_power

        # Status reassignment using cosine³(pitch) tolerance
        df_filtered = df[
            (df['sensor_10_avg'] <= 90) & 
            (df['sensor_10_avg'] >= -5)
        ].copy()
        df_filtered['status_before'] = df_filtered['status_type_id']

        def reassign_status(row):
            wind_speed = row['wind_speed_61_avg']
            pitch_deg = row['sensor_10_avg']
            actual_power = row['power_62_avg_norm']
            current_status = row['status_before']

            if wind_speed <= 3:
                return 0

            ideal = get_ideal_power(wind_speed, cut_in, rated_speed, rated_power)
            pitch_rad = np.deg2rad(pitch_deg)
            threshold = ideal * np.cos(pitch_rad)**3 * 0.98

            if current_status == 4:
                if actual_power <= 0.005:
                    return 4
                return 0 if actual_power >= threshold else 5

            return 0 if actual_power >= threshold else 5

        df_filtered['status_type_id'] = df_filtered.apply(reassign_status, axis=1)

        # Update main df
        df.loc[df_filtered.index, 'status_type_id'] = df_filtered['status_type_id']

        # Pitch binning (for reference)
        pitch_bins = np.arange(-5, 90, 5)
        pitch_labels = [f"{i}-{i+5}°" for i in pitch_bins[:-1]]
        df_filtered['pitch_bin'] = pd.cut(df_filtered['sensor_10_avg'], bins=pitch_bins, labels=pitch_labels, right=False)

        # Save output
        output_path = os.path.join(output_dir, f"{dataset_id}_WindFarm_B.csv")
        df.to_csv(output_path, index=False)

        print(f"✅ Dataset {dataset_id} processed and saved.")

    except Exception as e:
        print(f"❌ Error processing dataset {dataset_id}: {e}")



🔄 Processing dataset 2...
✅ Dataset 2 processed and saved.

🔄 Processing dataset 7...
✅ Dataset 7 processed and saved.

🔄 Processing dataset 19...
✅ Dataset 19 processed and saved.

🔄 Processing dataset 21...
✅ Dataset 21 processed and saved.

🔄 Processing dataset 23...
✅ Dataset 23 processed and saved.

🔄 Processing dataset 27...
✅ Dataset 27 processed and saved.

🔄 Processing dataset 34...
✅ Dataset 34 processed and saved.

🔄 Processing dataset 52...
✅ Dataset 52 processed and saved.

🔄 Processing dataset 53...
✅ Dataset 53 processed and saved.

🔄 Processing dataset 74...
✅ Dataset 74 processed and saved.

🔄 Processing dataset 77...
✅ Dataset 77 processed and saved.

🔄 Processing dataset 82...
✅ Dataset 82 processed and saved.

🔄 Processing dataset 83...
✅ Dataset 83 processed and saved.

🔄 Processing dataset 86...
✅ Dataset 86 processed and saved.

🔄 Processing dataset 87...
✅ Dataset 87 processed and saved.
